# Keras モデルを作る．

In [ ]:
#loding ndarray
print(type(np.load('./saving_ndarray/x_train/all/x_train_all_only_positive.npy')))
print(np.load('./saving_ndarray/x_train/all/x_train_all_only_positive.npy'))

x_train_positive_all = np.load('./saving_ndarray/x_train/all/x_train_all_only_positive.npy')
y_train_all = np.load('./saving_ndarray/x_train/all/y_train_all_only_positive.npy')
x_train_all_5crossval = np.load('./saving_ndarray/x_train/all/x_train_all_ 5crossval_only_positive.npy')
y_train_all_5crossval = np.load('./saving_ndarray/x_train/all/y_train_all_5crossval_only_positive.npy')
#y_train_all_5crossval_v8-1_only_positive

In [ ]:
#31
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv1D, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Dropout
from tensorflow.keras.utils import to_categorical
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist, cifar10
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model
import datetime

from tensorflow.keras.callbacks import EarlyStopping

#for tentsorboard
import tensorflow.keras.callbacks
#import tensorflow.keras.backend.tensorflow_backend as KTF
import tensorflow.keras.backend as KTF

#Vurnum = 実験番号
Vernum = 89

import os

## CNN version01(check_point: EarlyStopping)
https://qiita.com/kenichiro-yamato/items/60affeb7ca9f67c87a17

https://www.self-study-blog.com/dokugaku/python-pandas-dataframe-make/<br>
https://note.nkmk.me/python-pandas-assign-append/<br>

In [ ]:
#訓練データ
x_train = x_train_all
y_train = y_train_all

#検証データ
x_val = x_train_all_5crossval 
y_val = y_train_all_5crossval 


#old_session = KTF.get_session()
#session = tf.Session('')
#KTF.set_session(session)
#KTF.set_learning_phase(1)

#modelの定義の前に重みのリセットが必要で，これ？
tf.compat.v1.Session()
#sess = tf.compat.v1.Session()

#モデル構築
model = Sequential()
model.add(Conv2D(filters=3, kernel_size=(14, 4), strides=(1, 1), padding='valid', 
                 data_format="channels_last", dilation_rate=(1, 1), activation='relu', 
                 use_bias=True, input_shape=(14, 8, 1), kernel_initializer='glorot_uniform', 
                 bias_initializer='zeros',  name='Conv2D_01'))
#model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None, 
#                       name='MaxPooling2D_01'))
model.add(Flatten(name='Flatten_01'))
#model.add(Dropout(0.50))
#model.add(Dense(units=10, name='Denese_02', activation='relu'))
#model.add(Dense(units=50, name='Dense_01', activation='relu')
#model.add(Dense(units=10, name='Denese_03', activation='relu'))
model.add(Dense(units=2, name='Dense_out'))
model.add(Activation('softmax'))

#モデルの構造を表示
model.summary()

#訓練（学習）プロセスの設定
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])



#バリデーションロスが減少した場合に，各エポック終了後，モデルの重みを保存します
#checkpointer = ModelCheckpoint(filepath='./weights.hdf5', verbose=0, save_best_only=True, save_format="h5")
### add for TensorBoard
erly_stpg = EarlyStopping(patience=10, verbose=1,monitor='val_loss' , mode = 'min') 
chkfile = './callbacks/' + '{0:2d}'.format(Vernum)+'_{epoch:02d}-{val_loss:.4f}.hdf5' #format関数
check_point = ModelCheckpoint(filepath = chkfile, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
check_point = [check_point]

log_dir = "log/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
cbks=[tensorboard_callback]

#訓練の実行 
NUM_EPOCHS=350
NUM_BATCH=32
#history = model.fit(x_train, y_train, batch_size=NUM_BATCH, epochs=NUM_EPOCHS, validation_split=0.2, shuffle = True ,
#                    callbacks=[erly_stpg , chck_pont], verbose = 0, )
history = model.fit(x_train, y_train, batch_size=NUM_BATCH, epochs=NUM_EPOCHS, 
                    validation_split=0.2, shuffle = True, verbose = 0,
                    callbacks=check_point,
                   validation_data=(x_val, y_val))

print(type(history))
print(type(history.history))
print(history.history.keys())
print(history.history['loss'])
print(history.history['acc'])
print(history.history['val_loss'])
print(history.history['val_acc'])

In [ ]:
'''
from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.models import load_model

tf_config = some_custom_config
sess = tf.Session(config=tf_config)
graph = tf.get_default_graph()

# IMPORTANT: models have to be loaded AFTER SETTING THE SESSION for keras! 
# Otherwise, their weights will be unavailable in the threads after the session there has been set
set_session(sess)
#model = load_model(...)

## csvファイルにモデルの学習評価を書き出す．

In [ ]:
df1 = pd.DataFrame(
    data={'loss':history.history['loss'], 
          'acc':history.history['acc'],
          'val_loss':history.history['val_loss'], 
          'val_acc':history.history['val_acc']
         }, 
    index=np.arange(0, 350)
)
 
df1

In [ ]:
i=4
j=3
k=0
m=128
n=350

df2 = pd.DataFrame(
    data={'Kernel':f'14x{i}',
         'Conv2D':str(j),
         'hidden01':str(k),
         'Batch_size':str(m),
         'epochs':str(n)}, 
    index=np.arange(4)
)
df2

In [ ]:
print(pd.concat([df1, df2], axis=1))
pd3 = pd.concat([df1, df2], axis=1)

In [ ]:
var = 0
filename = f'../predict/DFS_parameter/trial_{var}.csv'
df1.to_csv(filename)

In [ ]:
pd3.iloc[0:5, :]

## 結果をプロットする．

In [ ]:
'''
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png')

In [ ]:
plt.plot(range(1, NUM_EPOCHS+1), history.history['acc'], "-o")
plt.plot(range(1, NUM_EPOCHS+1), history.history['val_acc'], "-o")
plt.title('model accuracy')
plt.ylabel('accuracy')  # Y軸ラベル
plt.xlabel('epoch')  # X軸ラベル
plt.grid()
plt.legend(['acc', 'val_acc'], loc='best')
#plt.xlim(0,NUM_EPOCHS)
#plt.ylim(0.55,1.0)
plt.show()

In [ ]:
plt.figure()
plt.plot()
plt.savefig('./predict/img/CNN_learning_rate_acc_01.png')
plt.close('all')

In [ ]:
plt.plot(range(1, NUM_EPOCHS+1), history.history['loss'], "-o")
plt.plot(range(1, NUM_EPOCHS+1), history.history['val_loss'], "-o")
plt.title('model loss')
plt.ylabel('loss')  # Y軸ラベル
plt.xlabel('epoch')  # X軸ラベル
plt.grid()
plt.legend(['loss', 'val_loss'], loc='best')
#plt.xlim(0,NUM_EPOCHS)
#plt.ylim(0.0,1.0)
plt.show()

In [ ]:
plt.figure()
plt.plot()
plt.savefig('./predict/img/CNN_learning_rate_loss_01.png')
plt.close('all')

In [ ]:
#old_session = KTF.get_session()

#session = tf.Session('')
#KTF.set_session(session)
#KTF.set_learning_phase(1)


loss, acc = model.evaluate(x_val, y_val)
print(f"Test Loss: {loss*100}%")
print(f"Test Acc: {acc*100}%")

### add for TensorBoard
#KTF.set_session(old_session)

## saving the model.

In [ ]:
#saving the model.
save_dir = "./saving_model/"
model_name = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

## load model してみる．

https://www.tensorflow.org/guide/keras/save_and_serialize<br>
from tensorflow import keras model = keras.models.load_model('path/to/location')

In [ ]:
#model = load_model("./model_saving/20220724-203049")
#from tensorflow import keras 
from tensorflow.keras.models import load_model
#model = load_model("./model_saving/20220724-203049")
model = load_model("./saving_model/20220725-045012")
#./model_saving/20220725-045012 
#モデルの構造を表示
model.summary()


#訓練の実行 
NUM_EPOCHS=10
NUM_BATCH=128

In [ ]:
plt.plot(range(1, NUM_EPOCHS+1), history.history['acc'], "-o")
plt.plot(range(1, NUM_EPOCHS+1), history.history['val_acc'], "-o")
plt.title('model accuracy')
plt.ylabel('accuracy')  # Y軸ラベル
plt.xlabel('epoch')  # X軸ラベル
plt.grid()
plt.legend(['acc', 'val_acc'], loc='best')
#plt.xlim(0,NUM_EPOCHS)
#plt.ylim(0.95,1.0)
plt.show()

plt.plot(range(1, NUM_EPOCHS+1), history.history['loss'], "-o")
plt.plot(range(1, NUM_EPOCHS+1), history.history['val_loss'], "-o")
plt.title('model loss')
plt.ylabel('loss')  # Y軸ラベル
plt.xlabel('epoch')  # X軸ラベル
plt.grid()
plt.legend(['loss', 'val_loss'], loc='best')
#plt.xlim(0,NUM_EPOCHS)
#plt.ylim(0.95,1.0)
plt.show()

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir ./log02/

## 学習後のモデルのパラメータを確認する．

In [ ]:
"""
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
%tensorboard --logdir=~/log/
%tensorboard --logdir=log02/ --port 43876

UsageError: Line magic function `%tensorboard` not found.


In [ ]:
weights = model.get_weights()
print(weights)

In [ ]:
print(weights.shape())

In [ ]:
lay0 = model.layers[0] 

In [ ]:
print(lay0)

In [ ]:
print(lay0.get_weights())

In [ ]:
print(len(lay0.get_weights()))

In [ ]:
print(len(lay0.get_weights()[0]))

In [ ]:
print(lay0.get_weights()[0][0])

In [ ]:
print(len(lay0.get_weights()[0][0]))

In [ ]:
print(len(lay0.get_weights()[0][0][0]))

In [ ]:
lay1 = model.layers[1]

In [ ]:
print(lay1)

In [ ]:
print(lay1.get_weights())

In [ ]:
lay2 = model.layers[2]
print(lay2)

In [ ]:
print(lay2.get_weights())

In [ ]:
lay3 = model.layers[3]
print(lay3)

In [ ]:
print(len(lay3.get_weights()[0]))

In [ ]:
print(lay3.get_weights())

In [ ]:
lay4 = model.layers[4]
print(lay4)

In [ ]:
print(lay4.get_weights())